In [1]:
import pandas as pd
import numpy as np
from math import log2
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as mticker
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
from statsmodels.stats.inter_rater import fleiss_kappa
from statsmodels.stats import inter_rater as irr
from statsmodels.tsa.stattools import adfuller
import pingouin as pg
pd.options.mode.chained_assignment = None

In [2]:
def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

C:\Users\succa\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\succa\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
def get_Fame(arr):
    first_entry = np.where(arr != 0)[0][0]
    return np.median(arr[first_entry:])

In [4]:
def get_Fame_ave(arr):
    first_entry = np.where(arr != 0)[0][0]
    return np.mean(arr[first_entry:])

In [5]:
def cronbach_alpha(df):
    df_corr = df.corr()
    N = df.shape[1]
    rs = np.array([])
    for i, col in enumerate(df_corr.columns):
        sum_ = df_corr[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)
    
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    return cronbach_alpha

In [6]:
Mass_shooting_df = pd.read_excel("Data/Mass Shootings (08_1966-04_2021).xlsx", sheet_name = "Mass_Shootings")
Mass_shooting_df = Mass_shooting_df.drop_duplicates(subset=['Date'])
Mass_shooting_df = Mass_shooting_df.reset_index()
N = len(Mass_shooting_df)
Variables = ["Target_Group","Shooting_Location","Level_of_Security","Leakage"]
wo = np.zeros([N,N])
for i in range(N):
    for j in range(i):
        wo[i,j] = 1
for Variable in Variables:
    Mass_shooting_df = Mass_shooting_df.reindex(columns = Mass_shooting_df.columns.tolist() + [Variable+"_Surprisal"])

Laplace_alpha = 1
for index , Variable in enumerate(Variables):
    uniques = np.unique(Mass_shooting_df[Variable])
    L = len(uniques)
    C = np.zeros([N, int(L)])
    J = np.ones([N, int(L)])
    for i in range(N):
        for j in range(len(uniques)):
            C[i, j] = 1 if (Mass_shooting_df[Variable][i] == uniques[j]) else 0
    priorDis = np.dot(wo,C)
    choices = np.argwhere(C==1)[:,1]
    for i in range(1,N):
        history = priorDis[i,:]
        choice = priorDis[i,choices[i]]
        y = -log2((choice+Laplace_alpha)/(Laplace_alpha*L+np.sum(history)))
        Mass_shooting_df[Variable+"_Surprisal"][i] = y
Mass_shooting_df = Mass_shooting_df.iloc[1:]
Mass_shooting_df = Mass_shooting_df.reset_index()

In [7]:
#Mass_shooting_df=Mass_shooting_df.iloc[49:].reset_index(drop=True) ## use if you want to igonre first couple of shootings 
for Variable in Variables:
    fs = np.array(Mass_shooting_df.loc[Mass_shooting_df["FameSeeker_Silva"]][Variable+"_Surprisal"])
    nfs = np.array(Mass_shooting_df.loc[Mass_shooting_df["FameSeeker_Silva"]==False][Variable+"_Surprisal"])
    pval = mannwhitneyu(nfs, fs, alternative='less',method='auto')
    print(Variable,pval)

Target_Group MannwhitneyuResult(statistic=748.0, pvalue=1.3020568195344246e-06)
Shooting_Location MannwhitneyuResult(statistic=911.5, pvalue=2.7763339737812056e-05)
Level_of_Security MannwhitneyuResult(statistic=1858.0, pvalue=0.4366150194647307)
Leakage MannwhitneyuResult(statistic=998.5, pvalue=0.00011884180192234494)


In [8]:
fame_seeking_df = Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)]
Extended = fame_seeking_df[(fame_seeking_df["Used an Extended Magazine"]==1.0)]
NotExtended = fame_seeking_df[(fame_seeking_df["Used an Extended Magazine"]==0.0)]
print(mannwhitneyu(Extended["Target_Group_Surprisal"], NotExtended["Target_Group_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Shooting_Location_Surprisal"], NotExtended["Shooting_Location_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Level_of_Security_Surprisal"], NotExtended["Level_of_Security_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Leakage_Surprisal"], NotExtended["Leakage_Surprisal"], method='auto'))

MannwhitneyuResult(statistic=50.0, pvalue=0.9728669499836549)
MannwhitneyuResult(statistic=27.0, pvalue=0.09981818474333017)
MannwhitneyuResult(statistic=33.0, pvalue=0.22002652639797382)
MannwhitneyuResult(statistic=35.0, pvalue=0.27770679226192396)


In [9]:
Wikipedia_df = pd.read_excel("Data/Mass Shootings (08_1966-04_2021).xlsx", sheet_name = "Wikiepdia")
for col in Wikipedia_df.columns:
    if col != "DateTime":
        Wikipedia_df[col] = pd.to_numeric(Wikipedia_df[col])
Wikipedia_df["DateTime"] = pd.to_datetime(Wikipedia_df["DateTime"])

In [10]:
Mass_shooting_df["Fame"] = np.zeros([len(Mass_shooting_df)])
Mass_shooting_df["Fame_av"] = np.zeros([len(Mass_shooting_df)])

for i in range(len(Mass_shooting_df)):
    if (Mass_shooting_df["WikiPage_reference"][i]=="There's a wikipedia page but not on the data site") | (Mass_shooting_df["WikiPage_reference"][i]=="No_page") | (Mass_shooting_df["WikiPage_reference"][i]=="Corrupted"): 
        Mass_shooting_df["Fame"][i] = np.nan
        Mass_shooting_df["Fame_av"][i] = np.nan
    else:
        fame_score = get_Fame(Wikipedia_df[Mass_shooting_df["WikiPage_reference"][i]].values)
        Mass_shooting_df["Fame"][i] = fame_score
        fame_score_av = get_Fame_ave(Wikipedia_df[Mass_shooting_df["WikiPage_reference"][i]].values)
        Mass_shooting_df["Fame_av"][i] = fame_score_av

In [11]:
Correlation_df = Mass_shooting_df.dropna(subset=['Fame'])[["Fame","Fame_av","Injured","Fatalities","Target_Group_Surprisal",
                                                          "Shooting_Location_Surprisal","Level_of_Security_Surprisal",
                                                          "Leakage_Surprisal"]]

In [13]:
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Target_Group_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Shooting_Location_Surprisal', covar=['Fatalities','Injured'], method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Level_of_Security_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Leakage_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))

,n,r,CI95%,p-val
spearman,76,0.3895,"[0.18, 0.57]",0.0006


,n,r,CI95%,p-val
spearman,76,0.4229,"[0.22, 0.59]",0.0002


,n,r,CI95%,p-val
spearman,76,0.0197,"[-0.21, 0.25]",0.8674


,n,r,CI95%,p-val
spearman,76,0.0698,"[-0.16, 0.29]",0.5548


In [14]:
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Target_Group_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Shooting_Location_Surprisal', covar=['Fatalities','Injured'], method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Level_of_Security_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Leakage_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))

,n,r,CI95%,p-val
spearman,76,0.3935,"[0.18, 0.57]",0.0005


,n,r,CI95%,p-val
spearman,76,0.3832,"[0.17, 0.56]",0.0008


,n,r,CI95%,p-val
spearman,76,0.009,"[-0.22, 0.24]",0.9396


,n,r,CI95%,p-val
spearman,76,0.0733,"[-0.16, 0.3]",0.535


In [15]:
Correlation_df.corr(method='spearman')[["Injured","Fatalities"]]

,Injured,Fatalities
Fame,0.495703,0.523841
Fame_av,0.497051,0.525503
Injured,1.000000,0.438646
Fatalities,0.438646,1.000000
Target_Group_Surprisal,0.351313,0.076692
Shooting_Location_Surprisal,0.142410,0.014456
Level_of_Security_Surprisal,0.033823,-0.057878
Leakage_Surprisal,0.214937,0.266262


In [16]:
Correlation_df.corr(method=spearmanr_pval)[["Injured","Fatalities"]]

,Injured,Fatalities
Fame,0.000005,0.000001
Fame_av,0.000005,0.000001
Injured,1.000000,0.000074
Fatalities,0.000074,1.000000
Target_Group_Surprisal,0.001860,0.510235
Shooting_Location_Surprisal,0.219750,0.901362
Level_of_Security_Surprisal,0.771769,0.619454
Leakage_Surprisal,0.062240,0.020078


In [17]:
fame_seeking_df = Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)]
Correlation_FS_df = fame_seeking_df.dropna(subset=['Fame'])[["Fame","Fame_av","Injured","Fatalities","Target_Group_Surprisal",
                                                          "Shooting_Location_Surprisal","Level_of_Security_Surprisal",
                                                          "Leakage_Surprisal","Total Firearms Brought to the Scene"]]
display(Correlation_FS_df.corr(method="spearman")[["Injured","Fatalities","Total Firearms Brought to the Scene"]])
display(Correlation_FS_df.corr(method=spearmanr_pval)[["Injured","Fatalities","Total Firearms Brought to the Scene"]])

,Injured,Fatalities,Total Firearms Brought to the Scene
Fame,0.436984,0.799176,-0.037150
Fame_av,0.427686,0.796070,-0.019106
Injured,1.000000,0.251296,-0.167348
Fatalities,0.251296,1.000000,-0.174616
Target_Group_Surprisal,0.284091,-0.253624,0.253681
Shooting_Location_Surprisal,0.017571,-0.304506,0.332398
Level_of_Security_Surprisal,-0.114907,-0.172199,-0.081899
Leakage_Surprisal,0.273760,-0.006211,0.043519
Total Firearms Brought to the Scene,-0.167348,-0.174616,1.000000


,Injured,Fatalities,Total Firearms Brought to the Scene
Fame,0.069787,0.000069,0.883646
Fame_av,0.076651,0.000078,0.940019
Injured,1.000000,0.314474,0.506863
Fatalities,0.314474,1.000000,0.488310
Target_Group_Surprisal,0.253245,0.309860,0.309748
Shooting_Location_Surprisal,0.944830,0.219221,0.177754
Level_of_Security_Surprisal,0.649815,0.494441,0.746645
Leakage_Surprisal,0.271658,0.980486,0.863863
Total Firearms Brought to the Scene,0.506863,0.488310,1.000000


In [19]:
df_Target_group = pd.read_excel("Data/Mass Shootings (08_1966-04_2021).xlsx", sheet_name = "Alpha&Kappa_TargetGroup")
df_Target_group = df_Target_group.set_index('subject')
df_Target_group = df_Target_group.replace("School Community",3)
df_Target_group = df_Target_group.replace("Random",2)
df_Target_group = df_Target_group.replace("Past Interaction",1)
df_Target_group = df_Target_group.replace("Demographic Group",0)
Target_group = df_Target_group.to_numpy()
Target_group_Kappa = irr.fleiss_kappa(irr.aggregate_raters(Target_group)[0], method='rand')
Target_group_Alpha = cronbach_alpha(df_Target_group)

In [20]:
df_Level_of_Security = pd.read_excel("Data/Mass Shootings (08_1966-04_2021).xlsx", sheet_name = "Alpha&Kappa_LvlofSecurity")
df_Level_of_Security = df_Level_of_Security.set_index('subject')
df_Level_of_Security = df_Level_of_Security.replace("No-Security", 0)
df_Level_of_Security = df_Level_of_Security.replace("Low-Security", 1)
df_Level_of_Security = df_Level_of_Security.replace("High-Security", 2)
Level_of_Security = df_Level_of_Security.to_numpy()
Level_of_Security_Kappa = irr.fleiss_kappa(irr.aggregate_raters(Level_of_Security)[0], method='rand')
Level_of_Security_Alpha = cronbach_alpha(df_Level_of_Security)

In [21]:
print("Target Group Alpha: ", Target_group_Alpha)
print("Target Group Kappa: ", Target_group_Kappa)
print("Level of Security Alpha: ", Level_of_Security_Alpha)
print("Level of Security Kappa: ", Level_of_Security_Kappa)

Target Group Alpha:  0.8463925659227177
Target Group Kappa:  0.5822222222222221
Level of Security Alpha:  0.920137295063748
Level of Security Kappa:  0.6669230769230766


In [22]:
count = 0
for col in Wikipedia_df.columns:
    if col != "DateTime":
        arr = Wikipedia_df[col].values
        nonzeroarrTS = arr[np.where(arr != 0)[0][0]:]
        print(adfuller(nonzeroarrTS)[1])
        if adfuller(nonzeroarrTS)[1] > 0.05:
            count +=1
print(count)

0.0
5.4167899814727044e-20
0.0
6.741717043509473e-16
2.828670632253555e-23
4.5061028867105114e-12
0.0
2.2283441965692223e-07
2.0804671579393384e-06
0.0006037053086086647
2.911514254585465e-24
7.309415989851811e-15
3.036596768671548e-16
6.276087821767912e-29
2.4945406187211212e-06
1.2384912450099632e-06
4.792002725274795e-07
0.0
6.51427925927738e-24
1.2432970774371459e-14
7.347073407540972e-16
2.7870278986638254e-19
6.1749252968037914e-12
4.075993977801848e-27
3.2957804509158117e-07
3.4957371671137805e-22
9.054059303690498e-11
0.037020407933710156
9.497199304987138e-15
3.169365157553058e-14
8.705315180371625e-11
1.6964371746675632e-22
9.29962784476465e-09
4.415972457214874e-19
9.574802624322004e-22
5.705332574507773e-18
1.4596346068839057e-21
1.0554755702890227e-19
1.2981894002493864e-17
4.735830923816811e-17
4.1086187318434984e-16
2.3623976917342825e-06
9.853516231655982e-12
1.9471704795945538e-25
5.750204315391931e-25
8.540999942499207e-29
2.4662634209858775e-15
3.384402762389203e-27


In [23]:
get_Fame(Wikipedia_df["charles whitman combined"].values)

995.0